In [29]:
# pip install -U sentence_transformers

In [30]:
# pip install -U faiss-gpu

In [7]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import faiss

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import csv

In [14]:
df = pd.read_csv('synthetic_names_samples.csv')

In [15]:
doc_embeddings = model.encode(df['transcription'].tolist())
doc_embeddings = np.array(doc_embeddings).astype('float32') # doing this for faiss

In [40]:
dimension = doc_embeddings.shape[1] #384 in this case.
index = faiss.IndexFlatIP(dimension)

In [41]:
index.add(doc_embeddings)

In [42]:
question = "What happened to the patient called Michael Dunstan who takes hydrochlorothiazide for hypertension?"
question_embedding = model.encode([question]).astype('float32')

In [43]:
k = 3 # number of documents.
distances, indices = index.search(question_embedding, k)

In [44]:
for i, idx in enumerate(indices[0]):
    doc_index = df.iloc[idx]['index']
    doc_text = df.iloc[idx]['transcription']
    print(f"Index: {doc_index}, Distance: {distances[0][i]:.4f}, Document: {doc_text}")

Index: 13, Distance: 0.7119, Document: NAME : Michael Dunstan  PAST MEDICAL HISTORY:,  Significant for hypertension.  The patient takes hydrochlorothiazide for this.  She also suffers from high cholesterol and takes Crestor.  She also has dry eyes and uses Restasis for this.  She denies liver disease, kidney disease, cirrhosis, hepatitis, diabetes mellitus, thyroid disease, bleeding disorders, prior DVT, HIV and gout.  She also denies cardiac disease and prior history of cancer.,PAST SURGICAL HISTORY: , Significant for tubal ligation in 1993.  She had a hysterectomy done in 2000 and a gallbladder resection done in 2002.,MEDICATIONS: , Crestor 20 mg p.o. daily, hydrochlorothiazide 20 mg p.o. daily, Veramist spray 27.5 mcg daily, Restasis twice a day and ibuprofen two to three times a day.,ALLERGIES TO MEDICATIONS: , Bactrim which causes a rash.  The patient denies latex allergy.,SOCIAL HISTORY: , The patient is a life long nonsmoker.  She only drinks socially one to two drinks a month. 